# Import_ssl

<div class="alert alert-block alert-info" style="border-radius: 20px;">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>importing secure socket layer</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "#" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    
</div>

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
<h1>1</h1>
In the beginning, we are having some import statements that are actually providing us with the required modules and libraries.
for proper execution of our code
    <ul>
        <li> sys</li>
        <li> json</li>
        <li> getpass </li>
        <li> redfish</li>
    </ul>
</div>

In [3]:
import sys
import json
import getpass
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

<div class="alert alert-block alert-success" style="border-radius: 20px;">
    <h1>2</h1>
This cell gives overview about of the function SSL(secure socket layer)
    <br>
1. It will retrieves the URI for the security service, sends a <b>GET</b> request to retrieve the security service object, and then retrieves the URI for the HTTPS certificate object linked to from the security service object.
    <br>
2. Else it searches the Redfish API resource directory for an instance of a HPE HTTPS certificate object. If found, it retrieves the URI for the HTTPS certificate object and breaks out of the loop.
    <br>
3. If the uri is not none then it retrieves the HTTPS certificate import URI from the HTTPS certificate object and then sends a <b>POST</b> request to import an SSL certificate using the Redfish API. The SSL certificate is passed in the request body.
    <br>
4.Handling the error after inserting the iso image in slot 
    If status code is <b>400</b> the code attempts to retrieve and print extended error message .
    If status code is <b> 200 OK</b> then it will print error meassage and status code.
    Else the insertion is <b>success</b> then code print result in <b>JSON</b> format and aslo printing the import is successfully done.

</div>

In [4]:
def import_ssl(_redfishobj, ssl_file_path):
  # intialize to None
    https_cert_uri = None
    body = dict()
    #Retrieving resource directory instances using Redfish API client.
    resource_instances = get_resource_directory(_redfishobj)
    
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        security_service_uri = managers_members_response.obj.Oem.Hpe.Links['SecurityService']\
                                                                                    ['@odata.id']
        security_service_response = _redfishobj.get(security_service_uri)
        https_cert_uri = security_service_response.obj.Links['HttpsCert']['@odata.id']
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#HpeHttpsCert.' in instance['@odata.type']:
                https_cert_uri = instance['@odata.id']
                break

    if https_cert_uri:
        https_cert_import_uri = _redfishobj.get(https_cert_uri).obj['Actions']\
                                                ['#HpeHttpsCert.ImportCertificate']['target']
        body = dict()
        body["Action"] = "HpeHttpsCert.ImportCertificate"
        body["Certificate"] = ssl_cert
        resp = _redfishobj.post(https_cert_import_uri, body)
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            print("\nImporting SSL Certificate may take a few minutes...\n "\
                  "iLO will reset with new changes.\n")

<div class="alert alert-block alert-success" style="border-radius: 20px;">
<h1>3</h1>
<b>The Main function overview . </b> <br>
    <ul>
        <li>When you are running server locally or running in remotely connect using secured connection then you need to enter proper login cerdentials.</li>
        <li> Creating redfish object and trying to establish connection with server using redfish object.</li>
        <li>If server is down or Unreachable we get server not reachable message!</li>
        <li>Read the content of an SSL certificate file at <b>SSL_FILE_PATH</b>. </li>
        <li>Lastly, Redfishobj.logout logs us out of the system and ends the session.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    SSL_FILE_PATH = "certificate.txt"
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    with open(SSL_FILE_PATH, 'r') as csr_data:
        ssl_cert = csr_data.read()
        csr_data.close()

    if ssl_cert:
        import_ssl(REDFISHOBJ, ssl_cert)
    else:
        raise Exception("Invalid SSL certificate.\n")
    REDFISHOBJ.logout()


<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success" style="border-radius: 20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>